In [ ]:
%pip install -e /Users/Kelsey/code/tlm

In [ ]:
# Create .env file inside of tlm project directory with variables filled in
# Note that AWS variables are only required if calling Bedrock models

In [ ]:
import os
from dotenv import load_dotenv

ENV_FILE_PATH = "/Users/Kelsey/code/tlm/.env"

load_dotenv(ENV_FILE_PATH, override=True)

In [ ]:
# Normal Question-Answering workflow

In [ ]:
from tlm.api import inference

openai_args = { "messages": [ { "role": "user", "content": "how many R's in strawberry" } ] }
result = await inference(openai_args=openai_args)
result

In [ ]:
# QA workflow with custom evals

In [ ]:
from tlm.api import inference
from tlm.config.base import ConfigInput

openai_args = { "messages": [ { "role": "user", "content": "Explain machine learning" } ] }
result = await inference(
    openai_args=openai_args,
    config_input=ConfigInput(
        criteria={
            "clarity": "The response is clear and easy to understand.",
            "conciseness": "The response is concise and to the point.",
        },
    )
)

result

In [ ]:
# Constrain outputs QA workflow

In [ ]:
from tlm.api import inference
from tlm.config.base import ConfigInput
from tlm.config.presets import QualityPreset, ReasoningEffort

openai_args = { "messages": [ { "role": "user", "content": "Classify the tone of the Text as positive, negative, or neutral. Text: The Earth is a beautiful place." } ] }

result = await inference(
    openai_args=openai_args,
    config_input=ConfigInput(
        quality_preset=QualityPreset.HIGH,
        reasoning_effort=ReasoningEffort.MEDIUM,
        constrain_outputs=["positive", "neutral", "negative"],
    ),
)
result

In [ ]:
# RAG workflow

In [ ]:
from tlm.api import inference

openai_args = { "messages": [ 
    {"role": "system", "content": "Answer the User Prompt using the Context: Simple water bottle holds 28 oz." }, 
    {"role": "user", "content": "how much water can i put in simple water bottle" },
] }

result = await inference(
    openai_args=openai_args,
    context="Simple water bottle holds 28 oz.",
)
result

In [ ]:
# Structured outputs scoring

In [ ]:
%pip install --upgrade openai

In [ ]:
from tlm.api import inference
from tlm.config.presets import QualityPreset
from tlm.config.base import ConfigInput
from openai import OpenAI

json_schema = {
    "name": "calendar_event",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {
            "name": { "type": "string" },
            "date": { "type": "string" },
            "participants": { 
                "type": "array",
                "items": { "type": "string" },
            },
        },
        "additionalProperties": False,
        "required": [ "name", "date", "participants" ],
    },
}

response_format = {
    "type": "json_schema",
    "json_schema": json_schema,
}

openai_kwargs = {
    "model": "gpt-4.1-mini",
    "messages": [
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    "response_format": response_format,
}
client = OpenAI()
completion = client.chat.completions.parse(**openai_kwargs)

result = await inference(
    openai_args=openai_kwargs,
    response={
        "chat_completion": completion.model_dump(),
        "perplexity": 0.95,
    },
    config_input=ConfigInput(
        quality_preset=QualityPreset.HIGH,
        model="gpt-4.1-mini",
    ),
)

result

In [ ]:
# Generate TLM response using AWS Bedrock model

In [ ]:
from tlm.api import inference
from tlm.config.base import ConfigInput
from tlm.config.presets import QualityPreset

openai_kwargs = {
    "model": "claude-3.7-sonnet",
    "messages": [
        {"role": "user", "content": "what is the capital of france"},
    ],
}

result = await inference(
    openai_args=openai_kwargs,
    config_input=ConfigInput(
        quality_preset=QualityPreset.HIGH,
    ),
)

result



In [ ]:
# Generate TLM response using Gemini model

In [ ]:
from tlm.api import inference
from tlm.config.base import ConfigInput
from tlm.config.presets import QualityPreset

chat_kwargs = {
    "model": "gemini/gemini-flash-latest",
    "messages": [
        {"role": "user", "content": "what is the capital of france"},
    ],
}

result = await inference(
    openai_args=chat_kwargs,
    config_input=ConfigInput(
        quality_preset=QualityPreset.HIGH,
    ),
)

result

In [ ]:
# Generate TLM response using Deepseek model

In [ ]:
from tlm.api import inference
from tlm.config.base import ConfigInput
from tlm.config.presets import QualityPreset

chat_kwargs = {
    "model": "deepseek/deepseek-chat",
    "messages": [
        {"role": "user", "content": "name the 3rd month of the year alphabetically"},
    ],
}

result = await inference(
    openai_args=chat_kwargs,
    config_input=ConfigInput(
        quality_preset=QualityPreset.HIGH,
    ),
)

result

In [ ]:
# Generate TLM response using Vertex AI model

In [ ]:
from tlm.api import inference
from tlm.config.base import ConfigInput
from tlm.config.presets import QualityPreset

import os
import json
file_path = '/Users/Kelsey/vertex.json'

with open(file_path, 'r') as file:
    vertex_credentials = json.load(file)
vertex_credentials_json = json.dumps(vertex_credentials)

openai_kwargs = {
    "model": "vertex_ai/gemini-2.5-pro",
    "messages": [
        {"role": "user", "content": "name the 3rd month alphabetically"},
    ],
    "vertex_credentials": vertex_credentials_json
}
result = await inference(
    openai_args=openai_kwargs,
)
result